In [33]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from pathlib import Path

# MapBox token
token = "pk.eyJ1IjoiZnlubndpIiwiYSI6ImNrODk3YmF6MzAzcDczbWs5NXdhaGpyNzYifQ.vHweJb-1hjDeE21tTs7tGQ"

## Load data

In [34]:
DIR_base = Path.cwd().parent
DIR_data = Path.joinpath(DIR_base, "data")

In [35]:
data = pd.read_pickle(Path.joinpath(DIR_data, "processed", "temp_yearly_summary.pkl"))
stations = pd.read_pickle(Path.joinpath(DIR_data, "processed", "stations_hourly_temp_hist.pkl"))

## Extract data

In [36]:
lats = stations['latitude']
lons = stations['longitude']
ids = stations['station_id']
names = stations['name']
states = stations['state']
altitudes = stations['altitude']

## Assemble Plot

In [37]:
# figure
fig = go.Figure()

# marker
markers = go.scattermapbox.Marker(
    size=15,
    color=altitudes,
    colorscale='thermal',
    opacity=0.8
)

# trace
trace = go.Scattermapbox(
    lat=lats,
    lon=lons,
    mode='markers',
    marker=markers,
    text=altitudes
)

# mapbox
my_mapbox = {
    'accesstoken': token,
    'style': 'basic',
    'zoom': 5,
    'center': {
        'lon': 10.02,
        'lat': 51.13
    }
}


fig.add_trace(trace)

# layout
fig.update_layout(
    title="DWD Stations in Germany",
    mapbox=my_mapbox,
    autosize=True
)

fig.show()

In [38]:
data

quality                                               \
                        count      mean       std  min  25%  50%  75%  max   
station_id date                                                              
3          1950-12-31  6599.0  5.000000  0.000000  5.0  5.0  5.0  5.0  5.0   
           1951-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0  5.0   
           1952-12-31  8784.0  5.000000  0.000000  5.0  5.0  5.0  5.0  5.0   
           1953-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0  5.0   
           1954-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0  5.0   
...                       ...       ...       ...  ...  ...  ...  ...  ...   
15555      2016-12-31  5880.0  3.000000  0.000000  3.0  3.0  3.0  3.0  3.0   
           2017-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0  3.0   
           2018-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0  3.0   
           2019-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0  3.0   
           2020-12-31  2112.0  1.340909  0.752241  1.0  1.0  1.0  1.0  3.0   

                      temperature             ...             humidity  \
                            count       mean  ...   75%   max    count   
station_id date                               ...                        
3          1950-12-31      6599.0  11.426686  ...  16.7  30.7   6599.0   
           1951-12-31      8760.0   9.869224  ...  14.4  30.0   8760.0   
           1952-12-31      8784.0   9.136521  ...  14.4  33.6   8784.0   
           1953-12-31      8760.0  10.011142  ...  15.0  30.2   8760.0   
           1954-12-31      8760.0   9.182317  ...  14.0  30.4   8760.0   
...                           ...        ...  ...   ...   ...      ...   
15555      2016-12-31      5880.0  11.139813  ...  16.5  29.3   5880.0   
           2017-12-31      8760.0   8.539966  ...  14.7  29.6   8760.0   
           2018-12-31      8453.0   9.557766  ...  16.3  30.6   8453.0   
           2019-12-31      8752.0   9.012317  ...  14.7  31.2   8752.0   
           2020-12-31      2112.0   3.565341  ...   6.7  15.9   2112.0   

                                                                            
                            mean        std   min   25%   50%   75%    max  
station_id date                                                             
3          1950-12-31  78.097439  15.152475  28.0  68.0  81.0  90.0  100.0  
           1951-12-31  78.777397  14.854556  20.0  70.0  82.0  91.0  100.0  
           1952-12-31  79.392873  15.355372  25.0  70.0  83.0  92.0  100.0  
           1953-12-31  80.487215  15.029949  19.0  71.0  84.0  93.0  100.0  
           1954-12-31  78.620776  14.803428  22.0  70.0  82.0  90.0  100.0  
...                          ...        ...   ...   ...   ...   ...    ...  
15555      2016-12-31  79.442687  16.653778  26.0  68.0  83.0  94.0  100.0  
           2017-12-31  79.321918  16.621397  22.0  68.0  82.0  94.0  100.0  
           2018-12-31  75.302969  17.154372  21.0  62.0  78.0  90.0  100.0  
           2019-12-31  75.058844  18.321588  16.0  62.0  79.0  91.0  100.0  
           2020-12-31  70.356061  17.090034  24.0  57.0  72.0  85.0   99.0  

[15661 rows x 24 columns]

In [108]:
# iterrows returns tuple!!!
# store everything in a dictionary

for station, frame in data.groupby(level=0):
    df = frame['temperature']['max'].reset_index()
    
    print()

    station_id       date   max
0            3 1950-12-31  30.7
1            3 1951-12-31  30.0
2            3 1952-12-31  33.6
3            3 1953-12-31  30.2
4            3 1954-12-31  30.4
..         ...        ...   ...
57           3 2007-12-31  33.1
58           3 2008-12-31  30.9
59           3 2009-12-31  33.8
60           3 2010-12-31  36.2
61           3 2011-12-31  17.8

[62 rows x 3 columns]

    station_id       date   max
0           44 2007-12-31  30.9
1           44 2008-12-31  32.2
2           44 2009-12-31  35.0
3           44 2010-12-31  33.6
4           44 2011-12-31  30.7
5           44 2012-12-31  37.0
6           44 2013-12-31  34.2
7           44 2014-12-31  32.2
8           44 2015-12-31  35.2
9           44 2016-12-31  32.2
10          44 2017-12-31  32.6
11          44 2018-12-31  35.8
12          44 2019-12-31  36.4
13          44 2020-12-31  16.9

    station_id       date   max
0           52 1976-12-31  32.2
1           52 1977-12-31  28.5
2           52 

    station_id       date   max
0          294 2004-12-31  31.4
1          294 2005-12-31  32.1
2          294 2006-12-31  37.2
3          294 2007-12-31  34.7
4          294 2008-12-31  33.3
5          294 2009-12-31  35.8
6          294 2010-12-31  35.5
7          294 2011-12-31  32.6
8          294 2012-12-31  35.0
9          294 2013-12-31  35.0
10         294 2014-12-31  33.5
11         294 2015-12-31  36.5
12         294 2016-12-31  34.3
13         294 2017-12-31  33.7
14         294 2018-12-31  37.1
15         294 2019-12-31  37.8
16         294 2020-12-31  17.5

    station_id       date   max
0          298 1981-12-31  28.8
1          298 1982-12-31  31.6
2          298 1983-12-31  34.0
3          298 1984-12-31  29.0
4          298 1985-12-31  28.9
5          298 1986-12-31  31.3
6          298 1987-12-31  29.8
7          298 1988-12-31  27.3
8          298 1989-12-31  31.6
9          298 1990-12-31  32.0
10         298 1991-12-31  30.3
11         298 1992-12-31  35.0
12     

    station_id       date   max
0          553 1951-12-31  31.0
1          553 1952-12-31  33.5
2          553 1953-12-31  31.8
3          553 1954-12-31  29.4
4          553 1955-12-31  30.4
5          553 1956-12-31  29.6
6          553 1957-12-31  33.8
7          553 1958-12-31  29.6
8          553 1959-12-31  35.9
9          553 1960-12-31  29.3
10         553 1961-12-31  33.0
11         553 1962-12-31  28.0
12         553 1963-12-31  31.5
13         553 1964-12-31  34.6
14         553 1965-12-31  28.6
15         553 1966-12-31  30.8
16         553 1967-12-31  31.3
17         553 1968-12-31  32.4
18         553 1969-12-31  32.4
19         553 1970-12-31  30.2
20         553 1971-12-31  -6.2

    station_id       date   max
0          554 1971-12-31  30.8
1          554 1972-12-31  32.6
2          554 1973-12-31  32.4
3          554 1974-12-31  29.2
4          554 1975-12-31  31.7
5          554 1976-12-31  35.2
6          554 1977-12-31  29.8
7          554 1978-12-31  32.8
8      

   station_id       date   max
0         982 1976-12-31  33.1
1         982 1977-12-31  29.9
2         982 1978-12-31  23.5

    station_id       date   max
0          983 2006-12-31  14.4
1          983 2007-12-31  34.4
2          983 2008-12-31  32.4
3          983 2009-12-31  32.2
4          983 2010-12-31  34.5
5          983 2011-12-31  34.3
6          983 2012-12-31  34.2
7          983 2013-12-31  35.2
8          983 2014-12-31  34.0
9          983 2015-12-31  35.8
10         983 2016-12-31  31.6
11         983 2017-12-31  32.4
12         983 2018-12-31  34.4
13         983 2019-12-31  35.6
14         983 2020-12-31  18.7

    station_id       date   max
0          991 2005-12-31  32.3
1          991 2006-12-31  33.9
2          991 2007-12-31  34.0
3          991 2008-12-31  30.3
4          991 2009-12-31  29.5
5          991 2010-12-31  32.5
6          991 2011-12-31  29.1
7          991 2012-12-31  36.6
8          991 2013-12-31  33.7
9          991 2014-12-31  32.2
10        

[70 rows x 3 columns]

    station_id       date   max
0         1473 2006-12-31  34.6
1         1473 2007-12-31  33.8
2         1473 2008-12-31  31.3
3         1473 2009-12-31  31.3
4         1473 2010-12-31  33.8
5         1473 2011-12-31  32.1
6         1473 2012-12-31  35.9
7         1473 2013-12-31  34.7
8         1473 2014-12-31  32.8
9         1473 2015-12-31  36.2
10        1473 2016-12-31  33.1
11        1473 2017-12-31  34.0
12        1473 2018-12-31  34.4
13        1473 2019-12-31  16.3

    station_id       date   max
0         1490 1965-12-31  30.3
1         1490 1966-12-31  30.3
2         1490 1967-12-31  30.4
3         1490 1968-12-31  32.1
4         1490 1969-12-31  29.6
5         1490 1970-12-31  30.2
6         1490 1971-12-31  32.0
7         1490 1972-12-31  30.8
8         1490 1973-12-31  30.4
9         1490 1974-12-31  30.0
10        1490 1975-12-31  31.8
11        1490 1976-12-31  31.7
12        1490 1977-12-31  33.4

   station_id       date   max
0        1503 20

...

    station_id       date   max
0         4411 2002-12-31  26.2
1         4411 2004-12-31  32.0
2         4411 2005-12-31  34.9
3         4411 2006-12-31  35.2
4         4411 2007-12-31  34.3
5         4411 2008-12-31  32.0
6         4411 2009-12-31  35.8
7         4411 2010-12-31  34.7
8         4411 2011-12-31  33.0
9         4411 2012-12-31  36.8
10        4411 2013-12-31  36.4
11        4411 2014-12-31  34.7
12        4411 2015-12-31  38.2
13        4411 2016-12-31  34.2
14        4411 2017-12-31  33.8
15        4411 2018-12-31  36.2
16        4411 2019-12-31  39.6
17        4411 2020-12-31  18.4

    station_id       date   max
0         4445 1989-12-31  18.7
1         4445 1990-12-31  30.6
2         4445 1991-12-31  28.2
3         4445 1992-12-31  11.2
4         4445 2003-12-31  28.7
5         4445 2004-12-31  22.8
6         4445 2005-12-31  28.6
7         4445 2006-12-31  31.5
8         4445 2007-12-31  30.7
9         4445 2008-12-31  28.5
10        4445 2009-12-31  29.9
11     

    station_id       date   max
0         4928 1977-12-31  29.8
1         4928 1978-12-31  28.9
2         4928 1979-12-31  30.5
3         4928 1980-12-31  30.9
4         4928 1981-12-31  31.1
5         4928 1982-12-31  30.4
6         4928 1983-12-31  36.1
7         4928 1984-12-31  36.6
8         4928 1985-12-31  34.2
9         4928 1986-12-31  32.7
10        4928 1987-12-31  31.2
11        4928 1988-12-31  31.7
12        4928 1989-12-31  34.5
13        4928 1990-12-31  33.2
14        4928 1991-12-31  35.5
15        4928 1992-12-31  34.3
16        4928 1993-12-31  32.4
17        4928 1994-12-31  33.0
18        4928 1995-12-31  33.1
19        4928 1996-12-31  31.2
20        4928 1997-12-31  32.1
21        4928 1998-12-31  35.3
22        4928 1999-12-31  33.3
23        4928 2000-12-31  33.2
24        4928 2001-12-31  34.8
25        4928 2002-12-31  35.6
26        4928 2003-12-31  37.9
27        4928 2004-12-31  32.0
28        4928 2005-12-31  33.5
29        4928 2006-12-31  34.8
30      

[64 rows x 3 columns]

    station_id       date   max
0         5516 1996-12-31  30.5
1         5516 1997-12-31  29.5
2         5516 1998-12-31  30.1
3         5516 1999-12-31  28.1
4         5516 2000-12-31  29.8
5         5516 2001-12-31  29.6
6         5516 2002-12-31  30.5
7         5516 2003-12-31  27.5
8         5516 2004-12-31  24.4
9         5516 2005-12-31  29.6
10        5516 2006-12-31  30.9
11        5516 2007-12-31  27.6
12        5516 2008-12-31  26.2
13        5516 2009-12-31  31.7
14        5516 2010-12-31  29.4
15        5516 2011-12-31  25.7
16        5516 2012-12-31  28.4
17        5516 2013-12-31  27.4
18        5516 2014-12-31  27.2
19        5516 2015-12-31  28.2
20        5516 2016-12-31  31.1
21        5516 2017-12-31  26.4
22        5516 2018-12-31  31.7
23        5516 2019-12-31  31.9
24        5516 2020-12-31  13.3

    station_id       date   max
0         5538 2005-12-31  31.9
1         5538 2006-12-31  33.2
2         5538 2007-12-31  33.5
3         5538 2

    station_id       date   max
0         6163 2001-12-31  31.1
1         6163 2002-12-31  31.4
2         6163 2003-12-31  31.4
3         6163 2004-12-31  27.7
4         6163 2005-12-31  30.6
5         6163 2006-12-31  31.6
6         6163 2007-12-31  32.9
7         6163 2008-12-31  30.2
8         6163 2009-12-31  31.7
9         6163 2010-12-31  31.8
10        6163 2011-12-31  28.6
11        6163 2012-12-31  33.3
12        6163 2013-12-31  32.4
13        6163 2014-12-31  28.6
14        6163 2015-12-31  30.9
15        6163 2016-12-31  30.3
16        6163 2017-12-31  27.9
17        6163 2018-12-31  34.5
18        6163 2019-12-31  32.0
19        6163 2020-12-31  15.5

    station_id       date   max
0         6170 2004-12-31  31.3
1         6170 2005-12-31  32.9
2         6170 2006-12-31  36.6
3         6170 2007-12-31  37.3
4         6170 2008-12-31  33.9
5         6170 2009-12-31  31.9
6         6170 2010-12-31  36.2
7         6170 2011-12-31  31.8
8         6170 2012-12-31  36.0
9      

13        7364 2020-12-31  17.2

    station_id       date   max
0         7367 2007-12-31  13.0
1         7367 2008-12-31  32.2
2         7367 2009-12-31  34.3
3         7367 2010-12-31  34.8
4         7367 2011-12-31  31.0
5         7367 2012-12-31  34.4
6         7367 2013-12-31  34.1
7         7367 2014-12-31  32.3
8         7367 2015-12-31  35.7
9         7367 2016-12-31  33.5
10        7367 2017-12-31  32.6
11        7367 2018-12-31  35.6
12        7367 2019-12-31  36.8
13        7367 2020-12-31  17.0

    station_id       date   max
0         7368 2007-12-31  11.8
1         7368 2008-12-31  31.6
2         7368 2009-12-31  33.0
3         7368 2010-12-31  33.4
4         7368 2011-12-31  32.2
5         7368 2012-12-31  32.5
6         7368 2013-12-31  32.7
7         7368 2014-12-31  31.4
8         7368 2015-12-31  35.8
9         7368 2016-12-31  33.0
10        7368 2017-12-31  32.0
11        7368 2018-12-31  35.6
12        7368 2019-12-31  35.9
13        7368 2020-12-31  16.2

    s

In [73]:
df = data.reset_index()

station_id       date quality                                          \
                              count      mean       std  min  25%  50%  75%   
0              3 1950-12-31  6599.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
1              3 1951-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
2              3 1952-12-31  8784.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
3              3 1953-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
4              3 1954-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
...          ...        ...     ...       ...       ...  ...  ...  ...  ...   
15656      15555 2016-12-31  5880.0  3.000000  0.000000  3.0  3.0  3.0  3.0   
15657      15555 2017-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0   
15658      15555 2018-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0   
15659      15555 2019-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0   
15660      15555 2020-12-31  2112.0  1.340909  0.752241  1.0  1.0  1.0  1.0   

            ... temperature       humidity                                    \
       max  ...         75%   max    count       mean        std   min   25%   
0      5.0  ...        16.7  30.7   6599.0  78.097439  15.152475  28.0  68.0   
1      5.0  ...        14.4  30.0   8760.0  78.777397  14.854556  20.0  70.0   
2      5.0  ...        14.4  33.6   8784.0  79.392873  15.355372  25.0  70.0   
3      5.0  ...        15.0  30.2   8760.0  80.487215  15.029949  19.0  71.0   
4      5.0  ...        14.0  30.4   8760.0  78.620776  14.803428  22.0  70.0   
...    ...  ...         ...   ...      ...        ...        ...   ...   ...   
15656  3.0  ...        16.5  29.3   5880.0  79.442687  16.653778  26.0  68.0   
15657  3.0  ...        14.7  29.6   8760.0  79.321918  16.621397  22.0  68.0   
15658  3.0  ...        16.3  30.6   8453.0  75.302969  17.154372  21.0  62.0   
15659  3.0  ...        14.7  31.2   8752.0  75.058844  18.321588  16.0  62.0   
15660  3.0  ...         6.7  15.9   2112.0  70.356061  17.090034  24.0  57.0   

                          
        50%   75%    max  
0      81.0  90.0  100.0  
1      82.0  91.0  100.0  
2      83.0  92.0  100.0  
3      84.0  93.0  100.0  
4      82.0  90.0  100.0  
...     ...   ...    ...  
15656  83.0  94.0  100.0  
15657  82.0  94.0  100.0  
15658  78.0  90.0  100.0  
15659  79.0  91.0  100.0  
15660  72.0  85.0   99.0  

[15661 rows x 26 columns]

In [32]:
df['station_id'] = 1

In [30]:
df

station_id       date quality                                          \
                              count      mean       std  min  25%  50%  75%   
0              1 1950-12-31  6599.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
1              1 1951-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
2              1 1952-12-31  8784.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
3              1 1953-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
4              1 1954-12-31  8760.0  5.000000  0.000000  5.0  5.0  5.0  5.0   
...          ...        ...     ...       ...       ...  ...  ...  ...  ...   
15656          1 2016-12-31  5880.0  3.000000  0.000000  3.0  3.0  3.0  3.0   
15657          1 2017-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0   
15658          1 2018-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0   
15659          1 2019-12-31  8760.0  3.000000  0.000000  3.0  3.0  3.0  3.0   
15660          1 2020-12-31  2112.0  1.340909  0.752241  1.0  1.0  1.0  1.0   

            ... temperature       humidity                                    \
       max  ...         75%   max    count       mean        std   min   25%   
0      5.0  ...        16.7  30.7   6599.0  78.097439  15.152475  28.0  68.0   
1      5.0  ...        14.4  30.0   8760.0  78.777397  14.854556  20.0  70.0   
2      5.0  ...        14.4  33.6   8784.0  79.392873  15.355372  25.0  70.0   
3      5.0  ...        15.0  30.2   8760.0  80.487215  15.029949  19.0  71.0   
4      5.0  ...        14.0  30.4   8760.0  78.620776  14.803428  22.0  70.0   
...    ...  ...         ...   ...      ...        ...        ...   ...   ...   
15656  3.0  ...        16.5  29.3   5880.0  79.442687  16.653778  26.0  68.0   
15657  3.0  ...        14.7  29.6   8760.0  79.321918  16.621397  22.0  68.0   
15658  3.0  ...        16.3  30.6   8453.0  75.302969  17.154372  21.0  62.0   
15659  3.0  ...        14.7  31.2   8752.0  75.058844  18.321588  16.0  62.0   
15660  3.0  ...         6.7  15.9   2112.0  70.356061  17.090034  24.0  57.0   

                          
        50%   75%    max  
0      81.0  90.0  100.0  
1      82.0  91.0  100.0  
2      83.0  92.0  100.0  
3      84.0  93.0  100.0  
4      82.0  90.0  100.0  
...     ...   ...    ...  
15656  83.0  94.0  100.0  
15657  82.0  94.0  100.0  
15658  78.0  90.0  100.0  
15659  79.0  91.0  100.0  
15660  72.0  85.0   99.0  

[15661 rows x 26 columns]